In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的E2E ML：MLOps第2阶段：实验：开始使用Vertex AI Vizier

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_vizier.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_vizier.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab上运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_vizier.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>           
</table>
<br/><br/><br/>

## 概览

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端的生产MLOps。本教程涵盖了第二阶段：实验：开始使用Vertex AI Vizier。

### 目标

在本教程中，您将学习如何在使用`Vertex AI`进行训练时使用`Vertex AI Vizier`。

本教程使用以下Google Cloud ML服务：

- `Vertex AI Training`
- `Vertex AI Hyperparameter Tuning`
- `Vertex AI Vizier`

执行的步骤包括：

- 使用随机算法进行超参数调整。
- 使用Vizier（贝叶斯）算法进行超参数调整。
- 为Vizier研究建议试验并更新结果。

###建议

在Google Cloud上进行端到端的MLOps时，以下是在何时使用Vertex AI Vizier进行超参数调整的最佳实践：

**网格搜索**

您有少量的离散值组合。例如，您有以下两个超参数和离散值：

- batch size：\[16, 32, 64\]
- lr：\[0.001, 0.01, 0.1\]

组合的总数为9（3 x 3）。

**随机搜索**

您有一小组超参数，其中至少有一个是连续值。例如，您有以下两个超参数和范围：

- batch size：\[16, 32, 64\]
- lr：0.001 .. 0.1

**Vizier搜索**

您可以是以下情况之一：

- 有大量的超参数和离散值
- 巨大的连续搜索空间
- 多个目标。

数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。本教程中使用的数据集版本已经内置在TensorFlow中。训练好的模型用于预测房屋的中位价格，单位为1千美元。

成本

本教程使用Google Cloud的可计费组件：

- Vertex AI
- 云存储

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

安装所需的软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade $USER_FLAG -q google-cloud-aiplatform \
                                       google-vizier==0.0.4

重新启动内核

安装了额外的软件包后，您需要重新启动笔记本内核，这样它就能找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### 设置你的谷歌云项目

**无论你使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建一个帐户时，你会获得一个$300的免费学分用于支付计算/存储费用。

1. [确保为你的项目启用了结算功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。 

1. 如果你是在本地运行这个笔记本，你需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入你的项目ID。然后运行该单元格确保Cloud SDK 对这个笔记本中的所有命令使用正确的项目。

**注意**：Jupyter会将以`！`为前缀的行作为shell命令运行，并且会将以`$`为前缀的Python变量插入这些命令中。

**设置您的项目ID**

**如果您不知道您的项目ID**，您可以尝试使用 `gcloud` 来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改 `REGION` 变量，该变量在笔记本的其余部分中使用。以下是 Vertex AI 支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法在 Vertex AI 上使用多区域存储桶进行训练。并非所有区域都支持所有 Vertex AI 服务。

了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type:"string"}
if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在进行现场教程会话，您可能正在使用共享的测试账户或项目。为了避免用户在创建的资源之间出现名称冲突，您可以为每个实例会话创建一个UUID，并将其附加到您在本教程中创建资源的名称后面。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud账户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经完成了身份验证。

**如果您正在使用Colab**，运行下面的单元格，并且按照提示进行身份验证，通过oAuth认证您的账户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，点击**创建**。

在**授予此服务账户对项目的访问权限**部分，点击角色下拉列表。在筛选框中输入"Vertex"，并选择**Vertex管理员**。在筛选框中输入"Storage Object Admin"，并选择**存储对象管理员**。

点击创建。一个包含您密钥的JSON文件将下载到本地环境。

在下面的单元格中将您的服务账户密钥的路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行此单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的笔记本环境如何，都需要按照以下步骤操作。**

当您初始化用于 Python 的 Vertex SDK 时，需要指定一个云存储暂存桶。暂存桶是您的数据集和模型资源所关联的所有数据在会话之间保留的地方。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + UUID

只有你的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip
from google.cloud.aiplatform.vizier import Study, pyvizier

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化用于 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练设置硬件加速器。

将变量`TRAIN_GPU / TRAIN_NGPU`设置为使用支持 GPU 的容器映像和分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个 GPU 容器映像，为每个 VM 分配 4 个 Nvidia Telsa K80 GPU，您应该指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，请指定`(None, None)`以使用一个在 CPU 上运行的容器映像。

了解更多信息，请前往[这里](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)查看您所在地区的硬件加速器支持。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

设置预建立的容器

设置用于训练的预建立的Docker容器镜像。

- 将变量`TF`设置为容器镜像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。以下列表显示了一些可用的预建立图像：

要获取最新列表，请参阅[用于训练的预建立容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)

#### 设置机器类型

接下来，设置用于训练的机器类型。

- 将变量`TRAIN_COMPUTE`设置为配置用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
 - `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：以下不支持用于训练：*

 - `standard`：2个vCPUs
 - `highcpu`：2、4和8个vCPUs

*注意：您还可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

## 独立的Vertex AI Vizer服务

`Vizier`服务可以作为一个独立的服务，用于选择试验的下一组参数。

*注意：* 该服务不执行试验。您需要自己创建试验和执行。

了解更多关于[Vizier的使用](https://cloud.google.com/vertex-ai/docs/vizier/using-vizier)。

## Vertex AI超参数调整服务

以下示例演示了如何使用Vertex AI超参数调整服务和 `random` 搜索算法设置、执行和评估试验。

了解有关[超参数调整概述](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)的更多信息。

### 检查超参数调整包

#### 包布局

在开始超参数调整之前，你将看看一个Python包是如何为自定义超参数调整工作而组装的。解压后，包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到Docker镜像操作环境的说明。

文件 `trainer/task.py` 是执行自定义超参数调整工作的Python脚本。 *注意*，当我们在工作池规范中提到它时，我们用点(`trainer.task`)替代了目录斜杠，并去掉了文件后缀(`.py`)。

#### 包装配

在下面的单元格中，您将组装训练包。

In [ ]:
# Make folder for Python hyperparameter tuning script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow==2.5.0',\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration hyperparameter tuning script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 内容

在下一个单元格中，您可以编写超参数调优脚本 task.py 的内容。我不会详细讨论，只是让您浏览一下。总结如下：

- 解析当前试验的超参数设置的命令行参数。
- 从命令行中获取保存模型文件的目录（`--model_dir`），如果没有指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 下载并预处理波士顿房价数据集。
- 构建一个 DNN 模型。
- 在模型的构建和编译过程中使用每个密集层的单元数和学习率超参数值。
- 定义一个回调函数 `HPTCallback`，在每个 epoch 结束时获取验证损失（`on_epoch_end()`）并通过 `hpt.report_hyperparameter_tuning_metric()` 报告给超参数调优服务。
- 使用 `fit()` 方法训练模型，并指定一个回调函数，将验证损失报告给超参数调优服务。

In [ ]:
%%writefile custom/trainer/task.py
# Custom Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
from hypertune import HyperTune
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--decay', dest='decay',
                    default=0.98, type=float,
                    help='Decay rate')
parser.add_argument('--units', dest='units',
                    default=64, type=int,
                    help='Number of units.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)

# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(args.units, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(args.units, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr, decay=args.decay))
  return model


model = build_and_compile_dnn_model()

# Instantiate the HyperTune reporting object
hpt = HyperTune()

# Reporting callback
class HPTCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        global hpt
        hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='val_loss',
        metric_value=logs['val_loss'],
        global_step=epoch)

# Train the model
BATCH_SIZE = 16
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=BATCH_SIZE, validation_split=0.1, callbacks=[HPTCallback()])
model.save(args.model_dir)

将超参数调整脚本存储在您的云存储桶中

接下来，将超参数调整文件夹打包成压缩的 tar 文件，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_boston.tar.gz

### 准备您的机器规格

现在为您的自定义超参数调整作业定义机器规格。这会告诉 Vertex 需要为超参数调整提供什么类型的机器实例。
- `machine_type`：要提供的GCP实例类型 -- 例如，n1-standard-8。
- `accelerator_type`：硬件加速器的类型，如果有的话。在这个教程中，如果您之前设置了变量`TRAIN_GPU != None`，则表示您正在使用GPU；否则您将使用CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的定制超参数调整作业定义磁盘规格。这告诉Vertex每个机器实例为超参数调整提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD或标准。SSD 更快，标准更便宜。默认为SSD。
- `boot_disk_size_gb`：磁盘大小（GB）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 定义工作池规范

接下来，您需要定义自定义超参数调整作业的工作池规范。工作池规范将包括以下内容：

- `replica_count`：要配置的此机器类型的实例数。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。

- `python_package`：要安装在VM实例上的Python训练包以及要调用的Python模块，以及Python模块的命令行参数。

现在让我们更深入了解Python包规范：

- `executor_image_spec`：这是为您的自定义超参数调整作业配置的Docker镜像。

- `package_uris`：这是您的Python训练包的位置（URI）列表，需要安装在配置实例上。这些位置需要在Cloud Storage存储桶中。这些可以是单个Python文件或整个包的zip（存档）。在后一种情况下，作业服务将解压（解档）内容到Docker镜像中。

- `python_module`：要调用以运行自定义超参数调整作业的Python模块（脚本）。在本示例中，您将调用`trainer.task.py` - 请注意，不需要附加`.py`后缀。

- `args`：要传递给相应Python模块的命令行参数。在本示例中，您将设置：
   - `"--model-dir=" + MODEL_DIR`：存储模型工件的Cloud Storage位置。有两种方法可以告诉超参数调整脚本在哪里保存模型工件：
      - 直接：将Cloud Storage位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），或
      - 间接：服务将Cloud Storage位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您需要在作业规范中告诉服务模型工件位置。
   - `"--epochs=" + EPOCHS`：训练的时期数。
   - `"--steps=" + STEPS`：每个时期的步骤（批次）数。
   - `"--distribute=" + TRAIN_STRATEGY"`：用于单个或分布式超参数调整的超参数调整分发策略。
      - `"single"`：单个设备。
      - `"mirror"`：单个计算实例上的所有GPU设备。
      - `"multi"`：所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job_" + UUID
MODEL_DIR = "{}/{}".format(BUCKET_URI, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = False
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_URI + "/trainer_boston.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

创建自定义作业

使用`CustomJob`类来创建一个自定义作业，比如用于超参数调整，具有以下参数：

- `display_name`：自定义作业的人类可读名称。
- `worker_pool_specs`：相应虚拟机实例的规范。

In [ ]:
job = aip.CustomJob(display_name="boston_" + UUID, worker_pool_specs=worker_pool_spec)

## 创建超参数调整作业

使用`HyperparameterTuningJob`类创建一个超参数调整作业，具有以下参数：

- `display_name`：自定义作业的可读名称。
- `custom_job`：此自定义作业中的工作池规范适用于所有试验中创建的CustomJobs。
- `metrics_spec`：要优化的度量。字典键是度量id，由您的训练作业报告，字典值是优化度量的目标（`最小化`或`最大化`）。
- `parameter_spec`：要优化的参数。字典键是度量id，作为命令行关键字参数传递给您的训练作业，字典值是度量的参数规范。
- `search_algorithm`：要使用的搜索算法：`grid`，`random`和`None`。如果指定`None`，则使用`Vizier`服务（贝叶斯）。
- `max_trial_count`：要执行的最大试验次数。

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hpt_job = aip.HyperparameterTuningJob(
    display_name="boston_" + UUID,
    custom_job=job,
    metric_spec={
        "val_loss": "minimize",
    },
    parameter_spec={
        "lr": hpt.DoubleParameterSpec(min=0.001, max=0.1, scale="log"),
        "units": hpt.IntegerParameterSpec(min=4, max=128, scale="linear"),
    },
    search_algorithm="random",
    max_trial_count=6,
    parallel_trial_count=1,
)

运行超参数调整作业

使用`run()`方法执行超参数调整作业。

In [ ]:
hpt_job.run()

显示超参数调整作业试验结果

在超参数调整作业完成之后，属性`trials`将返回每次试验的结果。

In [ ]:
print(hpt_job.trials)

现在看看哪个试验是最好的：

In [ ]:
best = (None, None, None, 0.0)
for trial in hpt_job.trials:
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

print(best)

获得最佳模型

如果您使用了让服务告诉调整脚本在哪里保存模型工件的方法（`DIRECT = False`），那么最佳模型的模型工件将保存在：

    MODEL_DIR/<best_trial_id>/model

In [ ]:
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

### 删除超参数调优作业

方法'delete（）'将删除超参数调优作业。

In [ ]:
hpt_job.delete()

## Vertex AI超参数调整和Vertex AI Vizer服务的结合

以下示例演示了如何使用Vertex AI超参数调整服务和`Vizier`搜索服务设置、执行和评估试验。

## 创建自定义作业

使用`CustomJob`类来创建自定义作业，例如用于超参数调整，具有以下参数：

- `display_name`：自定义作业的人类可读名称。
- `worker_pool_specs`：对应虚拟机实例的规格。
- `base_output_dir`：用于存储模型工件的 Cloud Storage 位置。

In [ ]:
job = aip.CustomJob(
    display_name="boston_" + UUID,
    worker_pool_specs=worker_pool_spec,
    base_output_dir=MODEL_DIR,
)

## 创建超参数调整作业

使用类`HyperparameterTuningJob`来创建一个超参数调整作业，其中包括以下参数：

- `display_name`: 用于自定义作业的人类可读名称。
- `custom_job`: 此自定义作业中的工作池规范适用于所有试验中创建的CustomJobs。
- `metrics_spec`: 要优化的指标。字典键是metric_id，由您的训练作业报告，字典值是指标的优化目标('minimize'或'maximize')。
- `parameter_spec`: 要优化的参数。字典键是metric_id，作为命令行关键字参数传递给您的训练作业，字典值是指标的参数规范。
- `search_algorithm`: 要使用的搜索算法：`grid`、`random`和`None`。如果指定了`None`，则会使用`Vizier`服务（贝叶斯）。
- `max_trial_count`: 要执行的最大试验次数。

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hpt_job = aip.HyperparameterTuningJob(
    display_name="boston_" + UUID,
    custom_job=job,
    metric_spec={
        "val_loss": "minimize",
    },
    parameter_spec={
        "lr": hpt.DoubleParameterSpec(min=0.0001, max=0.1, scale="log"),
        "units": hpt.IntegerParameterSpec(min=4, max=512, scale="linear"),
    },
    search_algorithm=None,
    max_trial_count=12,
    parallel_trial_count=1,
)

使用`run()`方法执行超参数调整作业。

In [ ]:
hpt_job.run()

### 显示超参数调整工作试验结果

在超参数调整工作完成后，属性`trials`将返回每次试验的结果。

In [ ]:
print(hpt_job.trials)

现在看看哪个试验是最好的：### 最佳试验

In [ ]:
best = (None, None, None, 0.0)
for trial in hpt_job.trials:
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

print(best)

获取最佳模型

如果您使用了让服务告诉调整脚本在哪里保存模型工件的方法（`DIRECT = False`），那么最佳模型的模型工件将保存在：

    MODEL_DIR/<best_trial_id>/model

In [ ]:
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

! gsutil ls {BEST_MODEL_DIR}

删除超参数调整作业

方法'delete()'将删除超参数调整作业。

In [ ]:
hpt_job.delete()

## 独立的 Vertex AI 视觉服务

`Vizier` 服务可作为独立服务用于选择试验的下一组参数。

*注意:* 该服务不执行试验。您需要自行创建试验和执行。

了解更多关于 [使用 Vizier](https://cloud.google.com/vertex-ai/docs/vizier/using-vizier)。

### 指定用于建议试验参数的算法

首先，创建一个`StudyConfig`，并指定用于建议下一个试验的算法。

    GRID_SEARCH：网格搜索
    RANDOM_SEARCH：随机搜索
    ALGORITHM_UNSPECIFIED：Vizier 贝叶斯算法

In [ ]:
problem = pyvizier.StudyConfig()
problem.algorithm = pyvizier.Algorithm.RANDOM_SEARCH

### 创建一个研究

一项研究是一系列实验或试验，帮助您优化超参数或参数。

在下面的例子中，目标是最大化 y = x^2，其中 x 的范围在 \[-10, 10\]。这个例子只有一个参数，并使用一个易于计算的函数来演示如何使用Vizier。

首先，您需要将要最小化或最大化的指标作为列表指定给 `metric_information` 属性。然后使用 `add_XXX_params()` 方法为相应的数据类型指定研究的参数：

    - add_bool_param
    - add_categorical_param
    - add_discrete_param
    - add_float_param
    - add_int_param

您可以使用 `create_or_load()` 方法来创建这项研究。

In [ ]:
STUDY_DISPLAY_NAME = "xpow2" + UUID

problem.metric_information.append(
    pyvizier.MetricInformation(name="y", goal=pyvizier.ObjectiveMetricGoal.MAXIMIZE)
)

params = problem.search_space.select_root()
params.add_float_param("x", -10.0, 10.0, scale_type=pyvizier.ScaleType.LINEAR)

study = Study.create_or_load(display_name=STUDY_DISPLAY_NAME, problem=problem)

STUDY_NAME = study.name
print("STUDY_NAME: {}".format(STUDY_NAME))

### 获取大臣研究

您可以使用方法`list()`来获取研究。

In [ ]:
studies = Study.list()
print(studies[0].gca_resource)

获取建议试验

接下来，使用方法`suggest()`查询Vizier服务获取建议试验，使用以下键/值对：

- `count`: 要建议的试验数量。

这个调用是一个长时间运行的操作。响应对象的`result()`方法将等待调用完成。

In [ ]:
SUGGEST_COUNT = 1

trials = study.suggest(count=SUGGEST_COUNT)

print(trials)

# Get the trial ID of the first trial
TRIAL_ID = trials[0].name

### 评估结果
在收到您的试验建议之后，评估每个试验并将每个结果记录为一个测量值。

例如，如果您要优化的函数是y = x^2，那么您可以使用试验中建议的x值来评估该函数。使用建议值为0.1，该函数计算结果为y = 0.1 * 0.1，结果为0.01。

### 添加测量

在评估您的试验建议并获取测量结果后，将该测量结果添加到您的试验中。

使用以下指令来存储您的测量结果并发送请求。在本例中，将RESULT替换为测量结果。如果您要优化的函数是y = x^2，并且建议的x值为0.1，则结果为0.01。

In [ ]:
RESULT = 0.01

measurement = pyvizier.Measurement()
measurement.metrics["y"] = RESULT

trials[0].add_measurement(measurement)

删除大臣研究

方法 'delete()' 将删除该研究。

In [ ]:
study.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

- Cloud Storage 存储桶

In [ ]:
delete_bucket = False

if os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI